# Pipeline for Splitting and Annotating Orders

This notebook demonstrates how to use three models to form a pipeline for splitting the source and annotating pizza and drink orders.

## Step 1: Load the Models

First, we need to load the three models responsible for splitting the source, annotating pizza orders, and annotating drink orders.

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Load the models
splitter_model = DistilBertForSequenceClassification.from_pretrained("/path/to/splitter_model")
pizza_annotator_model = DistilBertForSequenceClassification.from_pretrained("/path/to/pizza_annotator_model")
drink_annotator_model = DistilBertForSequenceClassification.from_pretrained("/path/to/drink_annotator_model")

# Set the models to evaluation mode
splitter_model.eval()
pizza_annotator_model.eval()
drink_annotator_model.eval()

## Step 2: Preprocess the Input Data

Next, we need to preprocess the input data to prepare it for the models.

In [ ]:
def preprocess_input(input_text):
    return tokenizer(input_text, padding='max_length', truncation=True, return_tensors="pt")

# Example input
input_text = "a bottle of ice tea and a dr pepper and one large diet sprite"
preprocessed_input = preprocess_input(input_text)

## Step 3: Use the Splitter Model to Split the Source

Use the first model to split the source into pizza and drink orders.

In [ ]:
def split_source(preprocessed_input):
    with torch.no_grad():
        splitter_outputs = splitter_model(**preprocessed_input)
        splitter_predictions = torch.argmax(splitter_outputs.logits, dim=-1)
    
    # Example split (this should be replaced with actual logic based on the model's output)
    split_text = {
        "pizzas": [],
        "drinks": ["a bottle of ice tea", "a dr pepper", "one large diet sprite"]
    }
    return split_text

split_text = split_source(preprocessed_input)

## Step 4: Use the Pizza Annotator Model to Annotate Pizza Orders

Use the second model to annotate the pizza orders.

In [ ]:
def annotate_pizza_orders(pizza_orders):
    annotations = []
    for order in pizza_orders:
        with torch.no_grad():
            preprocessed_order = tokenizer(order, padding='max_length', truncation=True, return_tensors="pt")
            outputs = pizza_annotator_model(**preprocessed_order)
            predictions = torch.argmax(outputs.logits, dim=-1)
        annotations.append(predictions)
    return annotations

pizza_annotations = annotate_pizza_orders(split_text["pizzas"])

## Step 5: Use the Drink Annotator Model to Annotate Drink Orders

Use the third model to annotate the drink orders.

In [ ]:
def annotate_drink_orders(drink_orders):
    annotations = []
    for order in drink_orders:
        with torch.no_grad():
            preprocessed_order = tokenizer(order, padding='max_length', truncation=True, return_tensors="pt")
            outputs = drink_annotator_model(**preprocessed_order)
            predictions = torch.argmax(outputs.logits, dim=-1)
        annotations.append(predictions)
    return annotations

drink_annotations = annotate_drink_orders(split_text["drinks"])

## Step 6: Combine and Display the Results

Combine the annotations and display the results.

In [ ]:
results = {
    "pizzas": pizza_annotations,
    "drinks": drink_annotations
}

print("Pizza Annotations:", results["pizzas"])
print("Drink Annotations:", results["drinks"])